In [ ]:
import numpy as np
np.seterr(over='ignore') # np.exp(1000) returns float(inf) without warnings 
import graphs

### Constants

In [ ]:
g_Na    = 30
g_K_DR  = 15
g_Ca    = 10
g_K_Ca  = 15
g_K_AHP = 0.8
g_C     = 2.1
g_L     = 0.1

V_Na =  60
V_K  = -75
V_Ca =  80
V_L  = -60

## Original formulation

### function $m$

In [ ]:
def α_m(V_s):
    return 0.32 * (-46.9 - V_s) / (np.exp((-46.9 - V_s) / 4) - 1)

def β_m(V_s):
    return 0.28 * (V_s + 19.9) / (np.exp((V_s + 19.9) / 5) - 1)

def τ_m(U):
    return 1.0 / (α_m(U) + β_m(U))

def m_infinity(U):
    return α_m(U) / (α_m(U) + β_m(U))

### function $n$

In [ ]:
def α_n(V_s):
    return 0.016 * (-24.9 - V_s) / (np.exp((-24.9 - V_s) / 5) - 1)

def β_n(V_s):
    return 0.25 * np.exp(-1 - 0.025 * V_s)

def τ_n(U):
    return 1.0 / (α_n(U) + β_n(U))

def n_infinity(U):
    return α_n(U) / (α_n(U) + β_n(U))

### function $h$

In [ ]:
def α_h(V_s):
    return 0.128 * np.exp((-43 - V_s) / 18)

def β_h(V_s):
    return 4 / (1 + np.exp((-20 - V_s) / 5))

def τ_h(U):
    return 1.0 / (α_h(U) + β_h(U))

def h_infinity(U):
    return α_h(U) / (α_h(U) + β_h(U))

### function $s$

In [ ]:
def α_s(V_d):
    return 1.6 / (1 + np.exp(-0.072 * (V_d - 5)))

def β_s(V_d):
    return 0.02 * (V_d + 8.9) / (np.exp((V_d + 8.9)/5) - 1)

def τ_s(U):
    return 1.0 / (α_s(U) + β_s(U))

def s_infinity(U):
    return α_s(U) / (α_s(U) + β_s(U))

### function $c$

In [ ]:
def H(U):
    """Heaviside step function"""
    return 0.0 if U < 0 else 1.0        

def α_c(V_d):
    return ((1 - H(V_d + 10)) * np.exp((V_d + 50) / 11 - (V_d + 53.5)/27) / 18.975
            + H(V_d + 10) * (2 * np.exp((-53.5 - V_d)/ 27)))
            
def β_c(V_d):
    return (1 - H(V_d + 10)) * (2 * np.exp((-53.5 - V_d)/ 27) - α_c(V_d))

def τ_c(U):
    return 1.0 / (α_c(U) + β_c(U))

def c_infinity(U):
    return α_c(U) / (α_c(U) + β_c(U))

### function $q$

In [ ]:
def α_q(Ca):
    return min(0.00002 * Ca, 0.01)

def β_q(Ca):
    return 0.001

def τ_q(U):
    return 1.0 / (α_q(U) + β_q(U))

def q_infinity(U):
    return α_q(U) / (α_q(U) + β_q(U))

### function $χ$

In [ ]:
def χ(Ca):
    return min(Ca / 250, 1)

## Continuous Approximation

### function $c$

In [ ]:
c_infinity_orig, τ_c_orig = c_infinity, τ_c  # keeping original ones around

def τ_c(V_d):
    return 3.627 * np.exp(0.03704 * V_d)

def c_infinity(V_d):
    return (1 / (1 + np.exp((-10.1 - V_d) / 0.1016)))**0.00925

### function $q$

In [ ]:
q_infinity_orig, τ_q_orig = q_infinity, τ_q 

def τ_q(Ca):
    return 657.9 * np.exp(-0.02023 * Ca) + 301.8 * np.exp(-0.002381 * Ca)

def q_infinity(Ca):
    return 0.7894 * np.exp(0.0002726 * Ca) - 0.7292 * np.exp(-0.01672 * Ca)

### function $χ$

In [ ]:
χ_orig = χ

def χ(Ca):
    return (    1.073 * np.sin(0.003453 * Ca + 0.08095) 
            + 0.08408 * np.sin(0.01634  * Ca - 2.34)
            + 0.01811 * np.sin(0.0348   * Ca - 0.9918))

## Temporal update functions

In [ ]:
# def update_intensities():
#     """Update the intensities variables"""
    
#     I_Na    = g_Na * m_infinity(V_s)**2 * h(V_s - V_Na)
#     I_K_DR  = g_K_DR * n(V_s - V_K)
#     I_Ca    = g_Ca * s(V_d - V_Ca)**2
#     I_K_Ca  = g_K_Ca * c(χ(Ca) * (V_d - V_K))
#     I_K_AHP = g_K_AHP * q (V_d - V_s)
#     I_SD    = g_C * (V_d - V_s)
#     I_Leak  = g_L * (V - V_L)
    
#     return I_Na, I_K_DR, I_Ca, I_K_Ca, I_K_AHP, I_SD, I_Leak

In [ ]:
# class HH_fun:
#     """Hodgkin-Huxley function"""
    
#     def __init__(self, x_infinity, τ_x, V_0):
#         self.x_infinity = x_infinity
#         self.τ_x = τ_x
#         self.V = V_0
#         self.x = x_infinity(V_0) # steady state
        
#     def __call__(self, V):
#         self.V = V
#         return self.x
    
#     def integrate(self, dt):
#         self.x += dt * (self.x_infinity(self.V) - x) / self.τ_x(V)

## Figure 1

In [ ]:
V_d_s = np.linspace(-100, 100, 1001)
c_inf_orig_s = [c_infinity_orig(V_d) for V_d in V_d_s]
c_inf_s      = [c_infinity(V_d)      for V_d in V_d_s]
τ_c_orig_s = [τ_c_orig(V_d) for V_d in V_d_s]
τ_c_s      = [τ_c(V_d)      for V_d in V_d_s]

In [ ]:
graphs.fig1a(V_d_s, c_inf_orig_s, c_inf_s, τ_c_orig_s, τ_c_s, title='c_∞ and τ_c', 
             y1_range=[0.0, 1.05], y2_range=[0, 160])

In [ ]:
c_inf_err_abs = [abs(x_orig - x) for x_orig, x in zip(c_inf_orig_s, c_inf_s)]
τ_c_err_rel = [abs(t_orig - t)/t_orig for t_orig, t in zip(τ_c_orig_s, τ_c_s)]
graphs.fig1b(V_d_s, c_inf_err_abs, τ_c_err_rel, title='c_∞ and τ_c, approximation error')

In [ ]:
Ca_s = np.linspace(0, 500, 1001)
q_inf_orig_s = [q_infinity_orig(Ca_i) for Ca_i in Ca_s]
q_inf_s      = [q_infinity(Ca_i)      for Ca_i in Ca_s]
τ_q_orig_s = [τ_q_orig(Ca_i) for Ca_i in Ca_s]
τ_q_s      = [τ_q(Ca_i)      for Ca_i in Ca_s]

In [ ]:
graphs.fig1a(Ca_s, q_inf_orig_s, q_inf_s, τ_q_orig_s, τ_q_s, title='q_∞ and τ_q', 
             y1_range=[0.0, 1.05], y2_range=[0, 1000])

In [ ]:
q_inf_err_abs = [abs(x_orig - x) for x_orig, x in zip(q_inf_orig_s, q_inf_s)]
τ_q_err_rel = [abs(t_orig - t)/t_orig for t_orig, t in zip(τ_q_orig_s, τ_q_s)]
graphs.fig1b(Ca_s, q_inf_err_abs, τ_q_err_rel, title='q_∞ and τ_q, approximation error')

In [ ]:
χ_orig_s = [χ_orig(Ca_i) for Ca_i in Ca_s]
χ_s      = [χ(Ca_i)      for Ca_i in Ca_s]

In [ ]:
graphs.fig1a(Ca_s, χ_orig_s, χ_s, title='q_∞ and τ_q', y1_range=[0.0, 1.05])

In [ ]:
χ_err_abs = [abs(x_orig - x) for x_orig, x in zip(χ_orig_s, χ_s)]
graphs.fig1b(V_d_s, χ_err_abs, title='χ approximation error')